In [ ]:
# RTX 4090

In [1]:
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install datasets evaluate tensorboard transformers accelerate sentencepiece kagglehub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 85.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 159.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 108.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 35.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 85.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 216.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgra

In [ ]:
import torch
import pandas as pd
import numpy as np
import re
from matplotlib import pylab as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset
from datasets import concatenate_datasets
from datasets import DatasetDict
from datasets import ClassLabel

import os
os.environ["KAGGLE_USERNAME"] = "USERNAME"
os.environ["KAGGLE_KEY"] = "KAGGLE_KEY"

import kagglehub

In [3]:
# Download the latest version.
dataset_path = kagglehub.dataset_download("abdoukarimkandji/wolbanking77v1")
print(dataset_path)

100%|██████████| 1.02M/1.02M [00:00<00:00, 1.26MB/s]

Extracting files...
/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/4


In [4]:
# Set the Seed for Reproducible Results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [5]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [7]:
df_train = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/4/wolbanking77v1/5k_split/train/train.csv")

df_test = pd.read_csv(
    "/root/.cache/kagglehub/datasets/abdoukarimkandji/wolbanking77v1/versions/4/wolbanking77v1/5k_split/test/test.csv")

df_train = df_train[['input_wo', 'label']]
df_test = df_test[['input_wo', 'label']]

#removal of punctuation marks
df_train['input_wo_clean'] = df_train['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')
df_test['input_wo_clean'] = df_test['input_wo'].replace(regex=True, to_replace = '[^\\w\\s]', value = ' ')

# Apply lowercase
df_train['input_wo_clean'] = df_train['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_test['input_wo_clean'] = df_test['input_wo_clean'].apply(lambda x: " ".join(x.lower() for x in x.split()))

train_ds = Dataset.from_pandas(df_train[['input_wo_clean', 'label']], split="train")
test_ds = Dataset.from_pandas(df_test[['input_wo_clean', 'label']], split="test")

raw_dataset = DatasetDict({"train": train_ds, "test": test_ds})
# raw_dataset = raw_dataset.remove_columns("__index_level_0__")
raw_dataset =  raw_dataset.rename_column("input_wo_clean", "text") # to match Trainer
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1000
    })
})

In [8]:
# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, return_tensors="pt")

In [9]:
from transformers import AutoTokenizer

# Model id to load the tokenizer
model_id = "castorini/afriteva_v2_base"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
tokenized_dataset = raw_dataset.map(tokenize, batched=True,remove_columns=["text"])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
labels = []
for label in tokenized_dataset["train"]:
    labels.append(label['label'])
labels = list(set(labels))
len(labels)

77

In [12]:
new_features = tokenized_dataset['train'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['train'] = tokenized_dataset['train'].cast(new_features)
new_features = tokenized_dataset['test'].features.copy()
new_features['label'] = ClassLabel(names=list(set(labels)))
tokenized_dataset['test'] = tokenized_dataset['test'].cast(new_features)

Casting the dataset:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
from transformers import AutoModelForSequenceClassification

# Prepare model labels - useful for inference
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label)

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at castorini/afriteva_v2_base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

In [14]:
import evaluate
import numpy as np

# Metric Id
metric = evaluate.load("f1")

# Metric helper method
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits[0], axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [36]:
#del trainer
#torch.cuda.empty_cache()

In [15]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments

# Id for remote repository
repository_id = "afriteva_v2_base-banking77-wolof"

# Define training args
training_args = TrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8, # 8 16
    per_device_eval_batch_size=8, # 8
    learning_rate=2e-05,
	num_train_epochs=20, # 5
    warmup_ratio=0.1,
    weight_decay=0.01,
		# PyTorch 2.0 specifics 
    # bf16=True, # bfloat16 training 
	# torch_compile=True, # optimizations
    optim="adamw_torch_fused", # improved optimizer 
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=200,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
    remove_unused_columns=False

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [16]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,4.389700,4.349866,0.002988
2,4.338100,4.286944,0.009528
3,4.288000,4.192608,0.046557
4,4.123300,3.893630,0.115516
5,3.842400,3.389958,0.165293
6,3.455800,3.022929,0.237676
7,3.218000,2.708289,0.314901
8,2.893900,2.466588,0.364191
9,2.701800,2.235907,0.391960
10,2.412800,2.077779,0.431261


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=10000, training_loss=2.745369479370117, metrics={'train_runtime': 3703.4644, 'train_samples_per_second': 21.601, 'train_steps_per_second': 2.7, 'total_flos': 4.88763273216e+16, 'train_loss': 2.745369479370117, 'epoch': 20.0})

In [17]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 1.5501234531402588,
 'eval_f1': 0.5492369640514755,
 'eval_runtime': 12.1756,
 'eval_samples_per_second': 82.131,
 'eval_steps_per_second': 10.266,
 'epoch': 20.0}

In [18]:
# Save processor and create model card
tokenizer.save_pretrained(repository_id)

('afriteva_v2_base-banking77-wolof/tokenizer_config.json',
 'afriteva_v2_base-banking77-wolof/special_tokens_map.json',
 'afriteva_v2_base-banking77-wolof/spiece.model',
 'afriteva_v2_base-banking77-wolof/added_tokens.json',
 'afriteva_v2_base-banking77-wolof/tokenizer.json')

In [19]:
model_id = "/workspace/NLP_TASKS/afriteva_v2_base-banking77-wolof/checkpoint-10000"
tokenizer = AutoTokenizer.from_pretrained(repository_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, use_safetensors=True).to("cuda")

In [20]:
def predict(batch, model):
    input_ids = torch.tensor(batch['input_ids']).to(device).unsqueeze(0)
    # token_type_ids = torch.tensor(batch['token_type_ids']).to(device).unsqueeze(0)
    attention_mask = torch.tensor(batch['attention_mask']).to(device).unsqueeze(0)
    with torch.no_grad():
        logits = model(input_ids=input_ids, 
                       # token_type_ids=token_type_ids, 
                       attention_mask=attention_mask).logits
        predicted = torch.argmax(logits, dim=-1)[0]
    return predicted

In [20]:
tokenized_dataset["test"][0].keys()

dict_keys(['label', 'input_ids', 'attention_mask'])

In [21]:
model.to("cuda")
predictions = [predict(batch, model) for batch in tokenized_dataset["test"]]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [22]:
predictions[0]

tensor(3, device='cuda:0')

In [23]:
y_pred = []
for y in predictions:
    y_pred.append(labels[y])
y_pred

['topping up by card',
 'Refund not showing up',
 'failed transfer',
 'disposable card limits',
 'age limit',
 'transfer timing',
 'exchange charge',
 'verify my identity',
 'declined card payment',
 'verify source of funds',
 'getting spare card',
 'extra charge on statement',
 'declined card payment',
 'supported cards and currencies',
 'declined transfer',
 'cash withdrawal not recognised',
 'verify my identity',
 'wrong amount of cash received',
 'card payment wrong exchange rate',
 'terminate account',
 'card payment not recognised',
 'wrong amount of cash received',
 'balance not updated after cheque or cash deposit',
 'reverted card payment?',
 'card payment wrong exchange rate',
 'verify my identity',
 'transfer timing',
 'transfer not received by recipient',
 'Refund not showing up',
 'verify my identity',
 'verify my identity',
 'top up reverted',
 'card not working',
 'declined cash withdrawal',
 'pending cash withdrawal',
 'fiat currency support',
 'country support',
 'card

In [24]:
y_test = []
for y in tokenized_dataset["test"]['label']:
    y_test.append(labels[y])
y_test

['topping up by card',
 'Refund not showing up',
 'transfer fee charged',
 'get disposable virtual card',
 'card arrival',
 'card delivery estimate',
 'exchange charge',
 'card delivery estimate',
 'transaction charged twice',
 'verify source of funds',
 'order physical card',
 'extra charge on statement',
 'declined card payment',
 'apple pay or google pay',
 'top up reverted',
 'cash withdrawal not recognised',
 'verify my identity',
 'wrong amount of cash received',
 'card payment wrong exchange rate',
 'terminate account',
 'card payment not recognised',
 'wrong amount of cash received',
 'balance not updated after cheque or cash deposit',
 'reverted card payment?',
 'card payment wrong exchange rate',
 'why verify identity',
 'transfer timing',
 'cancel transfer',
 'request refund',
 'verify my identity',
 'verify my identity',
 'top up reverted',
 'virtual card not working',
 'wrong amount of cash received',
 'pending cash withdrawal',
 'fiat currency support',
 'country support'

In [25]:
# from sklearn import metrics
# y_test = test_df['label']
print(metrics.classification_report(y_test, y_pred))

                                                  precision    recall  f1-score   support

                           Refund not showing up       0.52      0.82      0.64        17
                                activate my card       0.69      0.69      0.69        16
                                       age limit       0.67      0.60      0.63        10
                         apple pay or google pay       0.92      0.79      0.85        14
                                     atm support       0.89      0.89      0.89         9
                                automatic top up       0.65      0.79      0.71        14
         balance not updated after bank transfer       0.38      0.29      0.33        17
balance not updated after cheque or cash deposit       0.52      0.71      0.60        17
                         beneficiary not allowed       0.36      0.27      0.31        15
                                 cancel transfer       0.47      0.57      0.52        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
metrics.f1_score(y_test, y_pred, average='weighted')

0.5492369640514754

In [27]:
metrics.accuracy_score(y_test, y_pred)

0.564

In [28]:
metrics.precision_score(y_test, y_pred, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.5753145517294981

In [29]:
metrics.recall_score(y_test, y_pred, average='weighted', zero_division=0)

0.564

In [30]:
def compute_metrics(y_true, y_pred):
    print("f1_score:", metrics.f1_score(y_true, y_pred, average='weighted'))
    print("precision_score:", metrics.precision_score(y_true, y_pred, average='weighted'))
    print("recall_score:", metrics.recall_score(y_true, y_pred, average='weighted'))

In [31]:
compute_metrics(y_test, y_pred)

f1_score: 0.5492369640514754
precision_score: 0.5753145517294981
recall_score: 0.564


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
